# 単眼深度推定モデルを学習する
このノートではいよいよ単眼深度推定モデルの学習を行う。
ここでは簡単のために、最もシンプルなLossとネットワークをもちいて大枠の学習の方法を見ていく。

## ハイパーパラメタ
まずはじめにハイパーパラメタを設定しておく

In [32]:
frame_inds = [0, -1]  # 隣接フレームの番号
epochs = 20


## ネットワーク
学習するネットワークは現在のフレームの深度を推定するものと、フレーム間の姿勢の変化を推定するものの２つである。<br>
ここではそれぞれdepth netとpose netと呼ぶ。
depth netは深度マップを推定するネットワークであり、pose netは回転（X軸,Y軸,Z軸）と並進（X,Y,Z）の合計6個の数値を推定するネットワークである。

### depth net
ここではネットワークの詳細に関心はないため、`segmentation_models_pytorch` をつかいU-Netを定義する。

In [22]:
import segmentation_models_pytorch as smp
import torch

depth_net = smp.Unet('resnet18', in_channels=3, encoder_weights="imagenet", classes=1, activation='sigmoid')

# 入出力確認
depth_net(torch.zeros(1, 3, 224, 224)).shape

torch.Size([1, 1, 224, 224])

### pose net

pose netに関してもネットワークの詳細には関心がないため、torchvisionの学習済みモデルを持ってきて、部分的にレイヤーを差し替える。<br>
pose netの入力は現在フレームと隣接フレームの２枚であり、出力はそれらのフレーム間での姿勢の変化量であるため、入力チャンネルは合計で6チャンネルとなる。

In [23]:
from torchvision.models import resnet18
from torch.nn import Linear, Conv2d

pose_net = resnet18(pretrained=True)
pose_net.conv1 = Conv2d(in_channels=3*2, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
pose_net.fc = Linear(in_features=512, out_features=6)

# 入出力確認
pose_net(torch.zeros(1, 3*2, 224, 224)).shape

torch.Size([1, 6])

ネットワークの定義は以上である。

## Loss関数
ここではPhotometric lossを定義する。<br>

In [24]:
import sys
sys.path.append("../../src")

In [28]:
import torch.nn.functional as F
from camera import PinholeCamera
from functools import lru_cache

class PhotometricLoss(torch.nn.Module):
    def __init__(self, frame_inds):
        super().__init__()
        self.frame_inds = frame_inds
        self.l1_loss = torch.nn.L1Loss()

    def forward(self, y, y_pred):
        image_target = y["rgb_0"]
        depth = y_pred[f"depth_0"]
        intrinsic = y["intrinsic_0"]
        loss = 0
        for idx in self.frame_inds:
            # target 同士は比較しない
            if idx == 0:
                continue
            image_source = y[f"rgb_{idx}"]
            extrinsic_src2tgt = y[f"extrinsic_{idx}"]
            image_warped = self.warp(image_source, depth, intrinsic, extrinsic_src2tgt)
            loss += self.l1_loss(image_warped, image_target)
        return loss

    def warp(self, image_source, depth, intrinsic, extrinsic_src2tgt):
        image_coords = self.create_image_coords(depth.shape)
        world_coords = PinholeCamera.image2world(image_coords, intrinsic, extrinsic_src2tgt, depth)
        # skip world2camera() because the coord is already camera coords here.
        image_coords = PinholeCamera.camera2image(world_coords, intrinsic)
        # normalize the coord to be in [-1, 1] for grid sampling
        image_coords[..., 0] = image_coords[..., 0] / image_coords.shape[1] * 2 - 1
        image_coords[..., 1] = image_coords[..., 1] / image_coords.shape[0] * 2 - 1
        grid = image_coords
        image_warped = F.grid_sample(image_source, grid)
        return image_warped
                
    @lru_cache(None)
    def create_image_coords(map_shape):
        # 以前と同様にmeshgridで画像座標を生成する
        xi = torch.arange(0, map_shape[1], 1)
        yi = torch.arange(0, map_shape[0], 1)
        coord_x, coord_y = torch.meshgrid(xi, yi, indexing="xy")
        image_coords = torch.stack([coord_x, coord_y], axis=-1)
        image_coords = image_coords.float()
        return image_coords

In [30]:
criterion = PhotometricLoss(frame_inds=frame_inds)

## Pose Utils
pose netが推定した姿勢の変化量を4x4の行列に変換するユーティリティ関数を定義する。

## training loop

In [34]:
from dataset import VKITTI2

train_dataset = VKITTI2(root_dir="../../data")

loading sequences


100%|██████████| 5/5 [00:09<00:00,  1.89s/it]


In [37]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True, drop_last=True)

In [40]:
from torchvision import transforms

depth_net.train()
pose_net.train()

transform = transforms.Compose(
    [
        transforms.Resize(125, 414),
        transforms.ToTensor(),
    ]
)

for i in range(epochs):
    for batch in train_dataloader:
        # predict depth
        depth_net(batch["rgb_0"])
        assert()


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[10, 375, 1242, 3] to have 3 channels, but got 375 channels instead